Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data Loader

In [ ]:
ls "/content/drive/MyDrive/Colab Notebooks/"

 data_loader.py            OpenGait-master/   Untitled0.ipynb   zavrsni.ipynb
'Gait Recognition.ipynb'   Untitled           Untitled3.ipynb


In [ ]:
cp "/content/drive/MyDrive/Colab Notebooks/data_loader.py" "/content/"

In [ ]:
from torch.utils.data import Dataset, DataLoader
from pathlib import Path
from PIL import Image
import torch
from torchvision import transforms
import torch.nn as nn
import torch.optim as optim
import sys

if torch.cuda.is_available():
    print("GPU available " + torch.cuda.torch.cuda.get_device_name())

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class GaitImage():
    def __init__(self, path, subject):
        self.image = Image.open(path)
        self.subject = subject

class GaitImageDataset(Dataset):
    def __init__(self, images):
        self.images = images

    def __len__(self):
        return len(self.images)

    def getSubjects(self):
        unique_subjects = set()

        for img in self.images:
            unique_subjects.add(img.subject)

        return unique_subjects


    def __getitem__(self, index):
        gait_image = self.images[index]
        image_PIL = gait_image.image

        transform_to_tensor = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.Grayscale(),
            transforms.ToTensor()
        ])

        image = transform_to_tensor(image_PIL).to(device)

        label = torch.tensor(int(gait_image.subject))

        return image, label


def read_args():
    arguments = sys.argv
    dataset_arg = arguments[1]

    if dataset_arg not in ["oumvlp", "casiab74"]:
        raise ValueError(dataset_arg + " not supported, only `oumvlp` and `casiab74` allowed")

    model_arg = arguments[2]

    if model_arg not in ["vgg16", "resnet18", "resnet50"]:
        raise ValueError(model_arg + " not supported, only `vgg16`, `resnet18` and `resnet50` allowed")

    learning_rate = float(arguments[3])

    if learning_rate < 0:
        raise ValueError("Learning rate must be positive")

    max_epochs = int(arguments[4])

    return dataset_arg, model_arg, learning_rate, max_epochs

def load_datasets(dataset, batch_size=16, shuffle=True):
    dataset_folder = Path("/content/" + dataset +"/ProcessedBySID/train").resolve()
    images = []

    for img in sorted(dataset_folder.iterdir()):
        images.append(GaitImage(img.as_posix(), img.name[:-8]))

    train_dataset = GaitImageDataset(images)
    train_subjects = train_dataset.getSubjects()

    dataset_folder = Path("/content/" + dataset +"/ProcessedBySID/query").resolve()
    images = []

    for img in sorted(dataset_folder.iterdir()):
        subject = img.name[:-8]
        # Only include query images with subjects that exist in the train dataset
        if subject in train_subjects:
            images.append(GaitImage(img.as_posix(), subject))

    test_dataset = GaitImageDataset(images)

    train_loader = DataLoader(
        train_dataset,
        batch_size,
        shuffle
    )

    test_loader = DataLoader(
        test_dataset,
        batch_size,
        shuffle
    )

    return test_dataset, train_dataset, train_loader, test_loader

def get_model(model_arg, num_classes):
    # Load model
    model = torch.hub.load('pytorch/vision:v0.10.0', model_arg)

    if model_arg == "vgg16":
        # Modify conv layer to work with grayscale images
        model.features[0] = nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1)

        # Get the number of input features to the original output layer
        in_features = model.classifier[6].in_features

        # Replace the last layer with a new fully connected layer
        model.classifier[6] = nn.Linear(in_features, num_classes)
    else:
        # Modify conv layer to work with grayscale images
        model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3)

        # Get the number of input features to the original output layer
        in_features = model.fc.in_features

        # Replace the last layer with a new fully connected layer
        model.fc = nn.Linear(in_features, num_classes)

    model.to(device)

    return model

def calculate_accuracy(outputs, labels):
    _, predicted = torch.max(outputs, 1)
    correct = (predicted == labels).sum().item()
    total = labels.size(0)
    accuracy = correct / total * 100
    return accuracy

def calculate_topk_accuracy(outputs, labels, k=1):
    _, predicted = torch.topk(outputs, k, dim=1)
    correct = torch.sum(predicted == labels.view(-1, 1))
    accuracy = correct.item() / labels.size(0) * 100
    return accuracy

def main():
    dataset, model_arg, lr, max_epochs = read_args()

    print("---------------------INFO--------------------")
    print("DATASET:       ", dataset)
    print("MODEL:         ", model_arg)
    print("EPOCHS:        ", max_epochs)
    print("LEARNING RATE: ", lr)

    test_dataset, train_dataset, train_loader, test_loader = load_datasets(dataset, 32, True)

    print("---------------------TRAIN--------------------")
    print("Dataset size:        ", len(train_loader.dataset))
    print("Unique classse size: ", len(train_dataset.getSubjects()))
    print("---------------------TEST---------------------")
    print("Dataset size:        ", len(test_loader.dataset))
    print("Unique classse size: ", len(test_dataset.getSubjects()))
    print("----------------------------------------------")

    model = get_model(model_arg, len(train_dataset.getSubjects()))

    # Loss function
    loss_fn = nn.CrossEntropyLoss()
    loss_fn.to(device)

    # Optimizer
    optimizer = optim.Adam(model.parameters(), lr, weight_decay=0.001)

    def train_one_epoch():
        running_loss = 0.0
        avg_acc = 0.0
        avg_top5 = 0.0

        # Here, we use enumerate(training_loader) instead of
        # iter(training_loader) so that we can track the batch
        # index and do some intra-epoch reporting
        for i, data in enumerate(train_loader):
            # Every data instance is an input + label pair
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            # Zero your gradients for every batch!
            optimizer.zero_grad()

            # Make predictions for this batch
            outputs = model(inputs)

            # Compute the loss and its gradients
            loss = loss_fn(outputs, labels)
            loss.backward()

            # Adjust learning weights
            optimizer.step()

            # Gather data and report
            last_loss = loss.detach().item()
            running_loss += last_loss

            # Calculate accuracy
            top1_accuracy = calculate_topk_accuracy(outputs, labels, k=1)
            top5_accuracy = calculate_topk_accuracy(outputs, labels, k=5)
            avg_acc += top1_accuracy
            avg_top5 += top5_accuracy

            if i % 50 == 0:
                print('     batch {} loss: {} top-1 accuracy: {:.2f}% top-5 accuracy: {:.2f}%'.format(i, last_loss, top1_accuracy, top5_accuracy))

        return running_loss / len(train_loader), avg_acc / len(train_loader), avg_top5 / len(train_loader)

    def test_one_epoch():
        running_loss = 0.0
        avg_acc = 0.0
        avg_top5 = 0.0

        # Here, we use enumerate(training_loader) instead of
        # iter(training_loader) so that we can track the batch
        # index and do some intra-epoch reporting
        for i, data in enumerate(test_loader):
            # Every data instance is an input + label pair
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Make predictions for this batch
            outputs = model(inputs)

            # Compute the loss and its gradients
            loss = loss_fn(outputs, labels)

            # Gather data and report
            last_loss = loss.detach().item()
            running_loss += last_loss

            # Calculate accuracy
            top1_accuracy = calculate_topk_accuracy(outputs, labels, k=1)
            top5_accuracy = calculate_topk_accuracy(outputs, labels, k=5)
            avg_acc += top1_accuracy
            avg_top5 += top5_accuracy
            
            if i % 50 == 0:
                print('     batch {} loss: {} top-1 accuracy: {:.2f}% top-5 accuracy: {:.2f}%'.format(i, last_loss, top1_accuracy, top5_accuracy))

        return running_loss / len(test_loader), avg_acc / len(test_loader), avg_top5 / len(test_loader)

    for epoch in range(max_epochs):
        print("-------------------EPOCH {}------------------".format(epoch + 1))

        # Make sure gradient tracking is on, and do a pass over the data
        model.train()
        avg_loss, accuracy, top5_accuracy = train_one_epoch()

        # Set the model to evaluation mode, disabling dropout and using population
        # statistics for batch normalization.
        model.eval()

        # Disable gradient computation and reduce memory consumption.
        with torch.no_grad():
            avg_vloss, vaccuracy, vaccuracy_top5 = test_one_epoch()

        print("------------------TRAIN------------------")
        print('Loss {} Accuracy: {:.2f}% Top-5 accuracy: {:.2f}%'.format(avg_loss, accuracy, top5_accuracy))
        print("------------------TEST-------------------")
        print('Loss {} Accuracy: {:.2f}% Top-5 accuracy: {:.2f}%'.format(avg_vloss, vaccuracy, vaccuracy_top5))

        if epoch % 5 == 0:
            # Save the model's state dictionary, optimizer state, and other information
            checkpoint = {
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'other_info': '...'
            }

            # Define the file path for saving the checkpoint
            checkpoint_path = dataset + "_" + model_arg + 'model_checkpoint.pth' + str(epoch)

            # Save the checkpoint
            torch.save(checkpoint, checkpoint_path)

if __name__ == "__main__":
    main()

KeyboardInterrupt: ignored

# CASIA-B

## Extract

In [ ]:
!unzip /content/drive/MyDrive/casiab.zip -d /content

Streaming output truncated to the last 5000 lines.
 extracting: /content/casiab74/train/000021/nm072_000002.png  
 extracting: /content/casiab74/train/000021/cl072_000001.png  
 extracting: /content/casiab74/train/000021/nm144_000001.png  
 extracting: /content/casiab74/train/000021/nm000_000006.png  
 extracting: /content/casiab74/train/000021/nm054_000002.png  
 extracting: /content/casiab74/train/000021/bg036_000001.png  
 extracting: /content/casiab74/train/000021/nm054_000004.png  
 extracting: /content/casiab74/train/000021/nm054_000006.png  
 extracting: /content/casiab74/train/000021/cl054_000002.png  
 extracting: /content/casiab74/train/000021/bg054_000001.png  
 extracting: /content/casiab74/train/000021/nm018_000002.png  
 extracting: /content/casiab74/train/000021/cl108_000002.png  
 extracting: /content/casiab74/train/000021/cl090_000002.png  
 extracting: /content/casiab74/train/000021/cl018_000001.png  
 extracting: /content/casiab74/train/000021/bg072_000002.png  
 ext

## Process

In [ ]:
from logging import currentframe
import shutil
import os
from pathlib import Path


sourceFolder = Path("casiab74").resolve()
destinationFolder = "/ProcessedBySID/"
totalFiles = 0
currentFile = 0

destPath = sourceFolder.as_posix() + destinationFolder
Path.mkdir(Path(destPath), exist_ok = True)

def ExtractWalkingStatus(name):
    # Normal
    if  "nm" in name: return "nm"
    # In a coat
    if  "cl" in name: return "cl"
    # WIth a bag
    if  "bg" in name: return "bg"
    print("Error - Unknown walkign status: " + name)

def ProcessFile(src: Path, sid, ws, va, sn):
    #print("Processing {}/{}: ".format(currentFile, totalFiles) + src.name + " -> " + subjectID + "_" + walkingStatus + "_" + viewAnge + "_" + sequenceNumber, flush=True)
    shutil.copy(src, destPath + "_".join([sid, ws, va, sn]) + ".png")

def ProcessFile(category, src: Path, sid, i):
    #print("Processing {}/{}: ".format(currentFile, totalFiles) + src.name + " -> " + subjectID + "_" + str(i).zfill(3), flush=True)
    shutil.copy(src, destPath + "/" + category + "/"  + "_".join([str(sid).zfill(6), str(i).zfill(3)]) + ".png")

class DataImg(object):
    def __init__(self, sid, ws, va, sn, cat):
        self.subjectID = sid
        self.walkingStatus = ws
        self.viewAngle = va
        self.sequenceNumber = sn
        self.category = cat


"""
CASIA-B format:

CASIA-B Folder
    gallery
        subjectID1
            nm000_000001.png
            nm180_000002.png
            ...

        subjectID2
            bg072_000001.png
            bg090_000002.png
            ...

        ...
    query
        ...
    train
        ...

Image name:
WalkingStatusViewAngle_SequenceNumber.png

Subject ID - 4 numbers
Walking status - nm (normal), cl (in a coat), bg (with a bag)
View angle - 3 numbers (in degrees 0 - 180)
Sequence number - 2 numbers
"""

# Current folder contains Query, Train, Gallery

# Count files
totalFiles = 0
for i in sourceFolder.rglob("*.png"):
    totalFiles += 1

for category in sourceFolder.iterdir():
    if not category.name in ["gallery", "query", "train"]: continue
    Path.mkdir(Path(destPath + "/" + category.name), exist_ok = True)

    subjectID = 0
    for subject in category.iterdir():
        #subjectID = subject.name.removeprefix("00")
        subjectCounter = 0
        for image in subject.iterdir():
            split = image.name.split("_")
            walkingStatus = ExtractWalkingStatus(split[0])
            viewAngle = split[0].removeprefix(walkingStatus)
            sequenceNumber = split[1].removesuffix(".png").removeprefix("000")

            ProcessFile(category.name, image, subjectID, subjectCounter)
            subjectCounter += 1
            #ProcessFile(image, subjectID, walkingStatus, viewAngle, sequenceNumber)
            currentFile +=1
        subjectID += 1

## Train
data_loader arguments: `dataset` `model` `learning_rate` `epochs`
- Datasets: `oumvlp`, `casiab74`
- Models: `vgg16`, `resnet18`, `resnet50`
- Learning rate: positive float
- Epochs: positive int

### VGGNet 16

In [ ]:
!python data_loader.py "casiab74" "vgg16" 0.00001 40

GPU available Tesla T4
---------------------INFO--------------------
DATASET:        casiab74
MODEL:          vgg16
EPOCHS:         40
LEARNING RATE:  1e-05
---------------------TRAIN--------------------
Dataset size:         8107
Unique classse size:  74
---------------------TEST---------------------
Dataset size:         3298
Unique classse size:  50
----------------------------------------------
Downloading: "https://github.com/pytorch/vision/zipball/v0.10.0" to /root/.cache/torch/hub/v0.10.0.zip
-------------------EPOCH 1------------------
  batch 1 loss: 0.004322022914886475 top-1 accuracy: 0.00% top-5 accuracy: 6.25%
------------------TRAIN------------------
Loss 0.004322022914886475 Accuracy: 1.12% Top-5 accuracy: 6.63%
------------------TEST-------------------
Loss 4.303371388092411 Accuracy: 0.08% Top-5 accuracy: 0.26%
-------------------EPOCH 2------------------
  batch 1 loss: 0.004296299457550049 top-1 accuracy: 3.12% top-5 accuracy: 6.25%
------------------TRAIN-----------

### ResNet 18

In [ ]:
!python data_loader.py "casiab74" "resnet18" 0.00001 40

Streaming output truncated to the last 5000 lines.
     batch 81 loss: 0.025809969753026962 top-1 accuracy: 100.00% top-5 accuracy: 100.00%
     batch 82 loss: 0.027407407760620117 top-1 accuracy: 100.00% top-5 accuracy: 100.00%
     batch 83 loss: 0.02021748758852482 top-1 accuracy: 100.00% top-5 accuracy: 100.00%
     batch 84 loss: 0.031351346522569656 top-1 accuracy: 100.00% top-5 accuracy: 100.00%
     batch 85 loss: 0.023820538073778152 top-1 accuracy: 100.00% top-5 accuracy: 100.00%
     batch 86 loss: 0.0483260378241539 top-1 accuracy: 100.00% top-5 accuracy: 100.00%
     batch 87 loss: 0.025661813095211983 top-1 accuracy: 100.00% top-5 accuracy: 100.00%
     batch 88 loss: 0.027997927740216255 top-1 accuracy: 100.00% top-5 accuracy: 100.00%
     batch 89 loss: 0.032713182270526886 top-1 accuracy: 100.00% top-5 accuracy: 100.00%
     batch 90 loss: 0.028819739818572998 top-1 accuracy: 100.00% top-5 accuracy: 100.00%
     batch 91 loss: 0.04076945781707764 top-1 accuracy: 100.00

### ResNet 50

In [ ]:
!python data_loader.py "casiab74" "resnet50" 0.0001 40

Streaming output truncated to the last 5000 lines.
     batch 132 loss: 0.535660982131958 top-1 accuracy: 87.50% top-5 accuracy: 100.00%
     batch 133 loss: 0.4634723961353302 top-1 accuracy: 90.62% top-5 accuracy: 100.00%
     batch 134 loss: 0.7211815714836121 top-1 accuracy: 81.25% top-5 accuracy: 100.00%
     batch 135 loss: 0.6989263892173767 top-1 accuracy: 87.50% top-5 accuracy: 100.00%
     batch 136 loss: 0.553748607635498 top-1 accuracy: 81.25% top-5 accuracy: 100.00%
     batch 137 loss: 0.5244726538658142 top-1 accuracy: 87.50% top-5 accuracy: 96.88%
     batch 138 loss: 0.44653505086898804 top-1 accuracy: 90.62% top-5 accuracy: 100.00%
     batch 139 loss: 0.8312274217605591 top-1 accuracy: 71.88% top-5 accuracy: 100.00%
     batch 140 loss: 0.4851207733154297 top-1 accuracy: 87.50% top-5 accuracy: 100.00%
     batch 141 loss: 0.5589163303375244 top-1 accuracy: 90.62% top-5 accuracy: 100.00%
     batch 142 loss: 0.5274959206581116 top-1 accuracy: 90.62% top-5 accuracy: 10

# OUMVLP

## Extract

In [ ]:
!unzip /content/drive/MyDrive/oumvlp.zip -d /content

Streaming output truncated to the last 5000 lines.
 extracting: /content/oumvlp/query/007616/015_000001.png  
 extracting: /content/oumvlp/query/007616/090_000001.png  
 extracting: /content/oumvlp/query/007616/195_000001.png  
 extracting: /content/oumvlp/query/007616/030_000001.png  
 extracting: /content/oumvlp/query/007616/255_000001.png  
 extracting: /content/oumvlp/query/007616/075_000001.png  
 extracting: /content/oumvlp/query/007616/180_000001.png  
 extracting: /content/oumvlp/query/007616/210_000001.png  
   creating: /content/oumvlp/query/008868/
 extracting: /content/oumvlp/query/008868/000_000001.png  
 extracting: /content/oumvlp/query/008868/225_000001.png  
 extracting: /content/oumvlp/query/008868/270_000001.png  
 extracting: /content/oumvlp/query/008868/240_000001.png  
 extracting: /content/oumvlp/query/008868/255_000001.png  
 extracting: /content/oumvlp/query/008868/210_000001.png  
   creating: /content/oumvlp/query/003118/
 extracting: /content/oumvlp/query/00

## Process

In [ ]:
from logging import currentframe
import shutil
import os
from pathlib import Path


sourceFolder = Path("oumvlp").resolve()
destinationFolder = "/ProcessedBySID/"
totalFiles = 0
currentFile = 0

destPath = sourceFolder.as_posix() + destinationFolder
Path.mkdir(Path(destPath), exist_ok = True)


def ProcessFile(src: Path, sid, ws, va, sn):
    #print("Processing {}/{}: ".format(currentFile, totalFiles) + src.name + " -> " + subjectID + "_" + walkingStatus + "_" + viewAnge + "_" + sequenceNumber, flush=True)
    shutil.copy(src, destPath + "_".join([sid, ws, va, sn]) + ".png")

def ProcessFile(category, src: Path, sid, i):
    print("Processing {} {}/{}: ".format(category, currentFile, totalFiles) + src.name + " -> " + "_".join([str(sid).zfill(6), str(i).zfill(3)]), flush=True)
    shutil.copy(src, destPath + "/" + category + "/" + "_".join([str(sid).zfill(6), str(i).zfill(3)]) + ".png")

class DataImg(object):
    def __init__(self, sid, ws, va, sn, cat):
        self.subjectID = sid
        self.walkingStatus = ws
        self.viewAngle = va
        self.sequenceNumber = sn
        self.category = cat


"""
OUMVLP format:

OUMVLP Folder
    gallery
        subjectID1
            000_000001.png
            180_000002.png
            ...

        subjectID2
            210_000001.png
            270_000002.png
            ...

        ...
    query
        ...
    train
        ...

Image name:
ViewAngle_SequenceNumber.png

Subject ID - 4 numbers
View angle - 3 numbers (in degrees 0 - 180)
"""

# Current folder contains Query, Train, Gallery

# Count files
totalFiles = 0
for i in sourceFolder.rglob("*.png"):
    totalFiles += 1

for category in sourceFolder.iterdir():
    if not category.name in ["gallery", "query", "train"]: continue
    Path.mkdir(Path(destPath + "/" + category.name), exist_ok = True)

    subjectID = 0
    for subject in category.iterdir():
        subjectCounter = 0
        for image in subject.iterdir():
            split = image.name.split("_")

            ProcessFile(category.name, image, subjectID, subjectCounter)

            subjectCounter += 1
            currentFile +=1
        subjectID += 1

Streaming output truncated to the last 5000 lines.
Processing train 123539/267386: 030_000001.png -> 004764_004
Processing train 123540/267386: 045_000001.png -> 004764_005
Processing train 123541/267386: 060_000001.png -> 004764_006
Processing train 123542/267386: 015_000001.png -> 004764_007
Processing train 123543/267386: 060_000000.png -> 004764_008
Processing train 123544/267386: 225_000001.png -> 004764_009
Processing train 123545/267386: 045_000000.png -> 004764_010
Processing train 123546/267386: 015_000000.png -> 004764_011
Processing train 123547/267386: 180_000000.png -> 004764_012
Processing train 123548/267386: 090_000001.png -> 004764_013
Processing train 123549/267386: 240_000000.png -> 004764_014
Processing train 123550/267386: 240_000001.png -> 004764_015
Processing train 123551/267386: 255_000000.png -> 004764_016
Processing train 123552/267386: 000_000001.png -> 004764_017
Processing train 123553/267386: 075_000001.png -> 004764_018
Processing train 123554/267386: 25

## Train
data_loader arguments: `dataset` `model` `learning rate` `epochs`
- Datasets: `oumvlp`, `casiab74`
- Models: `vgg16`, `resnet18`, `resnet50`
- Learning rate: positive float
- Epochs: positive int


### VGGNet 16

In [ ]:
!python data_loader.py "oumvlp" "vgg16" 0.00001 40

### ResNet 18

In [ ]:
!python data_loader.py "oumvlp" "resnet18" 0.0001 40

### ResNet 50

In [ ]:
!python data_loader.py "oumvlp" "resnet50" 0.0001 40